# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import re

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import original data and preprocessed data

In [3]:
#sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
#calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

In [4]:
sales_train_val = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\sales_train_validation.csv')
calendar = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Hobbies\Demand Pattern Categorization'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [8]:
len(sales_intermittent)

869

In [9]:
len(sales_lumpy)

231

In [10]:
len(sales_erratic)

99

In [11]:
len(sales_smooth)

1061

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [12]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9607,HOBBIES_2_045_CA_4_validation,HOBBIES_2_045,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9650,HOBBIES_2_088_CA_4_validation,HOBBIES_2_088,HOBBIES_2,HOBBIES,CA_4,CA,0,1,0,0,...,0,3,0,0,0,1,1,0,0,0
9651,HOBBIES_2_089_CA_4_validation,HOBBIES_2_089,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9671,HOBBIES_2_109_CA_4_validation,HOBBIES_2_109,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# User-defined functions to calculate Metrics

In [13]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)


def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_smooth.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [16]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_001_CA_1,HOBBIES_1_001_CA_2,HOBBIES_1_001_CA_3,HOBBIES_1_001_CA_4,HOBBIES_1_004_CA_1,HOBBIES_1_004_CA_2,HOBBIES_1_004_CA_3,HOBBIES_1_004_CA_4,HOBBIES_1_005_CA_1,...,HOBBIES_2_115_CA_3,HOBBIES_2_115_CA_4,HOBBIES_2_117_CA_2,HOBBIES_2_120_CA_2,HOBBIES_2_121_CA_1,HOBBIES_2_121_CA_3,HOBBIES_2_128_CA_3,HOBBIES_2_132_CA_1,HOBBIES_2_139_CA_2,HOBBIES_2_147_CA_3
0,11101,4.0,3.0,4.0,1.0,18.0,23.0,34.0,4.0,3.0,...,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0
1,11102,3.0,6.0,5.0,3.0,16.0,23.0,45.0,1.0,6.0,...,4.0,5.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0
2,11103,5.0,4.0,7.0,4.0,13.0,15.0,46.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0
3,11104,7.0,11.0,6.0,2.0,5.0,38.0,47.0,7.0,2.0,...,2.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,2.0
4,11105,1.0,1.0,6.0,5.0,14.0,35.0,46.0,9.0,7.0,...,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,4.0,4.0,8.0,1.0,10.0,33.0,24.0,4.0,8.0,...,4.0,2.0,2.0,2.0,0.0,0.0,3.0,0.0,2.0,0.0
269,11609,4.0,0.0,5.0,4.0,12.0,20.0,59.0,6.0,9.0,...,10.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,2.0
270,11610,3.0,7.0,6.0,6.0,13.0,42.0,43.0,7.0,3.0,...,2.0,4.0,1.0,1.0,0.0,0.0,6.0,0.0,1.0,1.0
271,11611,3.0,1.0,5.0,4.0,11.0,31.0,40.0,7.0,8.0,...,5.0,2.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0


In [17]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

# Run ARIMA model with best combination

In [18]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [19]:
len(list_pattern_py)

1061

In [20]:
best_combination = (0, 1, 1)

In [21]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    
    train_product = train_data[product]
    test_product = test_data[product]
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()
    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Product': [product for product_count in range(len(test_product))],
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOBBIES_1_001_CA_1
Progressing: 0.0 %
Currently Running Product: HOBBIES_1_004_CA_1
Progressing: 0.09 %
Currently Running Product: HOBBIES_1_005_CA_1
Progressing: 0.19 %
Currently Running Product: HOBBIES_1_006_CA_1
Progressing: 0.28 %
Currently Running Product: HOBBIES_1_007_CA_1
Progressing: 0.38 %
Currently Running Product: HOBBIES_1_008_CA_1
Progressing: 0.47 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOBBIES_1_009_CA_1
Progressing: 0.57 %
Currently Running Product: HOBBIES_1_010_CA_1
Progressing: 0.66 %
Currently Running Product: HOBBIES_1_012_CA_1
Progressing: 0.75 %
Currently Running Product: HOBBIES_1_013_CA_1
Progressing: 0.85 %
Currently Running Product: HOBBIES_1_014_CA_1
Progressing: 0.94 %
Currently Running Product: HOBBIES_1_015_CA_1
Progressing: 1.04 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOBBIES_1_016_CA_1
Progressing: 1.13 %
Currently Running Product: HOBBIES_1_017_CA_1
Progressing: 1.23 %
Currently Running Product: HOBBIES_1_019_CA_1
Progressing: 1.32 %
Currently Running Product: HOBBIES_1_020_CA_1
Progressing: 1.41 %
Currently Running Product: HOBBIES_1_021_CA_1
Progressing: 1.51 %
Currently Running Product: HOBBIES_1_022_CA_1
Progressing: 1.6 %
Currently Running Product: HOBBIES_1_023_CA_1
Progressing: 1.7 %


...

C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOBBIES_1_415_CA_4
Progressing: 98.68 %
Currently Running Product: HOBBIES_1_416_CA_4
Progressing: 98.77 %
Currently Running Product: HOBBIES_1_417_CA_4
Progressing: 98.87 %
Currently Running Product: HOBBIES_1_420_CA_4
Progressing: 98.96 %
Currently Running Product: HOBBIES_1_421_CA_4
Progressing: 99.06 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOBBIES_1_423_CA_4
Progressing: 99.15 %
Currently Running Product: HOBBIES_1_424_CA_4
Progressing: 99.25 %
Currently Running Product: HOBBIES_2_018_CA_4
Progressing: 99.34 %
Currently Running Product: HOBBIES_2_043_CA_4
Progressing: 99.43 %
Currently Running Product: HOBBIES_2_045_CA_4
Progressing: 99.53 %
Currently Running Product: HOBBIES_2_088_CA_4
Progressing: 99.62 %
Currently Running Product: HOBBIES_2_089_CA_4
Progressing: 99.72 %
Currently Running Product: HOBBIES_2_109_CA_4
Progressing: 99.81 %
Currently Running Product: HOBBIES_2_115_CA_4
Progressing: 99.91 %
This line of code took 0.6290553616666666 minutes


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

# Calculate metrics for optimal combination

In [22]:
# Start calculating metrics 
start = timer()

df_arima_metrics = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.02864187333333324 minutes


In [23]:
df_arima_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_001_CA_1,0.585179,58.5179,54.8034,111.3573,1.170359e+00,1.539396e+00
1,HOBBIES_1_001_CA_2,0.500000,66.6667,72.3211,148.6550,2.000000e+00,2.124428e+00
2,HOBBIES_1_001_CA_3,0.500000,44.4444,45.3160,33.9027,4.000000e+00,5.309023e+00
3,HOBBIES_1_001_CA_4,0.593952,39.5968,37.6526,57.0464,1.187904e+00,1.552777e+00
4,HOBBIES_1_004_CA_1,0.500000,15.3846,15.4385,14.0903,2.000000e+00,2.427344e+00
...,...,...,...,...,...,...,...
1056,HOBBIES_2_121_CA_3,inf,inf,200.0000,NaN,7.087911e-13,7.087911e-13
1057,HOBBIES_2_128_CA_3,0.500000,42.8571,44.4832,37.8002,1.500000e+00,1.791752e+00
1058,HOBBIES_2_132_CA_1,inf,inf,200.0000,NaN,7.897530e-23,7.897530e-23
1059,HOBBIES_2_139_CA_2,0.500000,100.0000,127.5087,43.1479,1.000000e+00,1.009347e+00


In [24]:
df_arima_metrics.MAPE.mean()

57.515451208791205

In [25]:
df_arima_metrics.RMSE.mean()

3.826492289799019

In [26]:
df_arima_metrics.to_csv('ARIMA_Smooth_Forecast_Metrics.csv')